#

In [1]:
# Load required libraries
import pandas as pd

Added `encoding='latin-1'` when loading csv files to prevent the following error:

`UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf3 in position 1147: invalid continuation byte`

In [11]:
# Load csv files
hoja_1 = pd.read_csv('salida.csv', encoding='latin-1')
hoja_2 = pd.read_csv('promociones.csv', encoding='latin-1')
dias_festivos = pd.read_csv('dias_festivos_guatemala_2021_2025.csv', encoding='latin-1')

In [13]:
# Convert date columns to uniform data type
hoja_1['fecha_edicion'] = pd.to_datetime(hoja_1['fecha_edicion'], format='%Y-%m-%d')

hoja_2['fecha_inicio'] = pd.to_datetime(hoja_2['fecha_inicio'], format='%Y-%m-%d')
hoja_2['fecha_fin'] = pd.to_datetime(hoja_2['fecha_fin'], format='%Y-%m-%d')

dias_festivos['Fecha'] = pd.to_datetime(dias_festivos['Fecha'], format='%Y-%m-%d')

In [14]:
# Get unique values for Tipo from hoja_2. These values will be used for
# generating additional columns in the output file
unique_tipo = list(set(hoja_2['Tipo']))

In [15]:
# Get unique dates from hoja 1
unique_dates = hoja_1['fecha_edicion'].unique()

Building a helper dataframe to help out

* First, a column of unique dates located in hoja_1
* Second, a column for 'Feriados', where a 1 will be set if the uniqe date matches to a date in dias_festivos dataframe
* Third, columns will be created for each item in unique_tipo, with values of 0

In [16]:
# Building helper data frame
helper_df = pd.DataFrame(
    {'date': unique_dates}
)

In [17]:
# Add 'Feriados' column to helper dataframe
helper_df['Feriados'] = helper_df['date'].isin(dias_festivos['Fecha']).astype(int)

In [18]:
# Add a column for each 'Tipo'
for tipo in unique_tipo:
    helper_df[tipo] = 0

Building another intermediate dataframe based on hoja_2 to make work easier

* Group by 'Tipo'
* Get lowest fecha_inicio and highest fecha_fin per contiguous time spans for 'Tipo'

In [19]:
# Helper function to collapse intervals. Given a list of intervals (tuple of start, end),
# merge overlaping and adjacent intervals
def collapse_intervals(intervals):
    # Sort intervals by start date
    intervals.sort(key=lambda x: x[0])
    collapsed = []

    for current_interval in intervals:
        if not collapsed:
            collapsed.append(list(current_interval))
        else:
            # Check if current interval starts within or right after the last merged interval
            # If so, merge them.
            if current_interval[0] <= collapsed[-1][1] + pd.Timedelta(days=1):
                # Merge the intervals by extending the end date if necessaru
                collapsed[-1][1] = max(collapsed[-1][1], current_interval[1])
            else:
                collapsed.append(list(current_interval))
    
    return collapsed

In [20]:
# Function to merge intervals for each group of p_type
def collapse_group(type_group):
    # Get group name
    group_name = type_group.name
    # Create a list of (start_Data, end_date) tuples for the group
    intervals = list(zip(type_group['fecha_inicio'], type_group['fecha_fin']))
    collapsed = collapse_intervals(intervals)

    collapsed_df = pd.DataFrame({
        'Tipo': [group_name] * len(collapsed),
        'fecha_inicio': [interval[0] for interval in collapsed],
        'fecha_fin': [interval[1] for interval in collapsed]
    })

    return collapsed_df

In [21]:
hoja_2_collapsed = hoja_2.groupby('Tipo', group_keys=False)[['fecha_inicio', 'fecha_fin']].apply(collapse_group)

Let's make some magic:

Now, for all the unique dates in helper_df, they will be checked to see if they fall in the range per 'Tipo' from the hoja_2_collapsed, and set a 1 in helper_df's corresponding column 

In [22]:
for idx, row in hoja_2_collapsed.iterrows():
    tipo_h = row['Tipo']
    start_date = row['fecha_inicio']
    end_date = row['fecha_fin']

    # Boolean mask to check if date is within range
    in_range = (helper_df['date'] >= start_date) & (helper_df['date'] <= end_date)

    # Set corresponding column to 1 where the condition is met
    helper_df.loc[in_range, tipo_h] = 1

The helper_df is complete. This df has for every unique date that appeared in hoja_1:
* A column with the dates
* A column for 'Feriados': set to 1 if it matches a date in the dias_festivos
* Columns for each unique Tipo from hoja 2: with it's corresponding Tipo set to 1 if there is a match in the date ranges from hoja_2


Now, the helper_df will be joined to hoja_1 to produce the expected outcome

In [23]:
# Pre-work: rename date column from helper_df to fecha edicion, in order to hava a cleaner result after merging
helper_df = helper_df.rename(columns={'date': 'fecha_edicion'})

In [24]:
# Merge the data
hoja_1_enhanced = hoja_1.merge(helper_df, how='left', on='fecha_edicion')

In [25]:
# Save to output file
hoja_1_enhanced.to_csv('hoja_1_enhanced.csv', index=False)